In [1]:
# Can we find parent/child word relationships faster
# One way to achieve this is to use the GPU via cupy
# cupy is basically numpy on the gpu. Let's start with a simple case

In [2]:
# standard libraries
from time import perf_counter_ns
import time

In [3]:

# external libraries
import pandas as pd
import numpy as np

# custom libraries
from _run_constants import *
from part_00_file_db_utils import *
from part_00_process_functions import *


# Load input data
The char_matrix and the formatted words

In [5]:
word_df, wg_df, letter_dict, char_matrix, \
    word_group_id_list, word_id_list, wchar_matrix = load_input_data(
        db_path=rc.DB_PATH, db_name=rc.DB_NAME,
        in_file_path=rc.IN_FILE_PATH)


...loading words into a dataframe...
...query execution took: 2.04 seconds...
...loading word groups into a dataframe...
...query execution took: 1.93 seconds...
...loading the letter dictionary...
...loading the char matrix...
...subsetting the char matrix...


# let's process 1000 rows using a single lookup on the full wchar_matrix

In [8]:
# load the total number of anagrams
n_possible_anagrams = load_possible_anagrams(db_path=rc.DB_PATH,
                                             db_name=rc.DB_NAME)

...query execution took: 0.01 seconds...


In [9]:
f"{n_possible_anagrams :,}"

'198,842,245'

# Using the CPU
Create a data extraction scenario that is very similar to Matrix Extraction Technique 1: Using the full matrix. I remove the excess code and create a simple loop. 

In [10]:
# sample 1000 rows
n_samples = 1000
sample_wg_id = wg_df['word_group_id'].sample(n = n_samples, random_state = 42).to_list()

# let's add a specific word: acanthology - the study of spines (as of sea urchins) especially as an adjunct of taxonomy
# this is to ensure that extraction technique produces the correct values
# On a complete run, there should by 26 parent words and 329 child words
# This will find the 26 parent words, and depending on what else in the sample,
# a number of child words

if 746 not in sample_wg_id:
    sample_wg_id.append(746)    

# create an output object
output_list = np.full(shape=(n_possible_anagrams, 2),
                          fill_value=-1, dtype=int)

row_count = 0
anagram_pair_count = 0

for i_wg_id, wg_id in enumerate(sample_wg_id):
    # identify parent words
    outcome = wchar_matrix - wchar_matrix[wg_id, ]
    
    # compute the score by finding where rows, across all columns, are GTE 0
    outcome_indices = np.all(outcome >= 0, axis=1)    

    n_from_words = outcome_indices.sum()

    if n_from_words >= 1:
        # extract anagrams based on index values    
        outcome_word_id_list = word_group_id_list[outcome_indices]    

        # we have matches
        # the focal word
        curr_output_list = np.zeros(shape=(n_from_words, 2), dtype=int)

        # update the output list with the word_id_list - these are from/parent words
        curr_output_list[:, 0] = outcome_word_id_list

        # update with the word_id - this is the to/child word
        curr_output_list[:, 1] = wg_id

        # enumerate the from/parent words
        new_anagram_pair_count = anagram_pair_count + n_from_words

        output_list[anagram_pair_count:new_anagram_pair_count,
                    :] = curr_output_list

        # n_to_word_counter = collections.Counter(output_list[:, 0])
        #intermediate_to_word_count.update(outcome_word_id_list.tolist())

        # set the anagram pair count
        anagram_pair_count = new_anagram_pair_count
    
    row_count += 1

    if row_count % 100 == 0:
        print(row_count)
print('truncating list')
output_indices = np.all(output_list >= 0, axis=1)
output_list = output_list[output_indices,]
del output_indices
output_list.shape

100
200
300
400
500
600
700
800
900
1000
truncating list


(367672, 2)

In [11]:
# approximately 43 seconds
from_word_counter = collections.Counter(output_list[:,1])
to_word_counter = collections.Counter(output_list[:,0])

In [ ]:
# the number of from word groups
# so, this technique is correct
from_word_counter[746]

26

In [ ]:
# Setup a similar workflow, using CuPy

# build a selector

In [ ]:
# let's split the matrix, gather the values for each split, and then combine
n_subset_letters = 3
wg_df["letter_selector"] = wg_df["letter_group_ranked"].str[:n_subset_letters]

letter_selector_list = wg_df["letter_selector"].unique()
letter_selector_list.sort()
letter_selector_id_dict = {ls: i_ls for i_ls, ls in enumerate(letter_selector_list)}

wg_df["letter_selector_id"] = wg_df["letter_selector"].map(letter_selector_id_dict)
# here's the thing: I need to be able to identify on a single matrix the rows that match various conditions.
# I can't step through it and create objects at abandon. 
# so, given our wchar_matrix: what are the rows that match to such and such?
# we can add three columns to track this... 

In [ ]:
wg_df.head()

In [ ]:
# load letter ranks
sql = 'select letter, total_letter_rank from letter_count;'

In [ ]:
lr_df = query_db(sql = sql, db_path=rc.DB_PATH, db_name=rc.DB_NAME)

In [ ]:
lr_dict = {l:r for l, r in zip(lr_df['letter'], lr_df['total_letter_rank'])}

In [ ]:
wg_df['n_records'] = int(1)

In [ ]:
col_names = ['letter_selector_id', 'letter_selector', 'n_records']

In [ ]:
ls_df = wg_df[col_names].groupby(col_names[:-1]).agg(ls_count = ('n_records', 'sum')).reset_index()

In [ ]:
ls_df.head()

In [ ]:
ls_df['ls_count'].sum()

In [ ]:
ls_df['ls_count'].describe()

In [ ]:
def get_ls_index(ls:str):
    return [letter_dict[l] for l in ls]

In [ ]:
# this is effectively a column selector
ls_df['ls_index'] = ls_df['letter_selector'].map(get_ls_index)

In [ ]:
ls_df.head()

In [ ]:
ls_df.head()

In [ ]:
n_possible_anagrams = load_possible_anagrams(db_path=rc.DB_PATH, db_name=rc.DB_NAME)

In [ ]:
# create the output list
output_list = np.full(shape = (n_possible_anagrams, 2), fill_value=-1)
output_time_list = []

# start counting
anagram_pair_count = 0

#for ls_id_index in range(0, 10):
for ls_row_id, ls_row in ls_df.iloc[:None].iterrows():    
    if ls_row_id % 100 == 0:
        print(ls_row_id)
    start_time = perf_counter_ns()
    
    # get letter selector id information
    ls_id = ls_row['letter_selector_id']
    ls_id_index = np.array(ls_row['ls_index'])    

    ##
    # BUILD A COLUMN SELECTOR
    ##
    # make sure that only values GTE 0 are selected so that the right number of
    # columns are return.
    #curr_ls_id = ls_id_index[ls_id_index >= 0]
    
    ##
    # SUBSET THE wchar_matrix by column selector
    ##    
    outcome_indices = np.all(wchar_matrix[:, ls_id_index] >= 1, axis=1)
    
    # this is the sub-matrix from which to query
    ls_wchar_matrix = wchar_matrix[outcome_indices, :]
        
    # this is the list of word group ids that correspond to the word group ids
    # in the ls_wchar_matrix
    temp_wg_id_list = word_group_id_list[outcome_indices]
    # place into a dictionary to go from wg_id to wg_index. What is the index
    # of wg_id 675?
    # wg_id_dict = {wg_id:wg_index for wg_index, wg_id in enumerate(temp_wg_id_list)}

    # this is the number of word groups that meet certain criteria. 
    # for example, words that feature the letters: 'bro'    
    n_search_space = temp_wg_id_list.shape[0]
        
    #def my_func(row):
    #    return temp_wg_id_list[np.all(a = (ls_wchar_matrix - ls_wchar_matrix[row, :]) >= 0, axis = 1)]

    #for ii in range(0, ls_wchar_matrix.shape[0]):    
    #for i_curr_wg_id, curr_wg_id in enumerate(temp_wg_id_list):
    # the current list of words featuring the set of least common letters.
    # these are the words have the least common letters of 'bro'    
    curr_wg_id_list = wg_df.loc[wg_df['letter_selector_id'] == ls_id, 'word_group_id'].to_numpy()
    # n_lookups = curr_wg_id_list.shape[0]
    # n_search_space >= n_lookups, always. 
    for i_curr_wg_id, curr_wg_id in enumerate(curr_wg_id_list):
    
        
        #temp_wg_id = wg_id_dict[curr_wg_id]
        temp_wg_id = np.where(temp_wg_id_list == curr_wg_id)[0][0]
        #print(curr_wg_id, temp_wg_id)

        #outcome_word_id_list = my_func(row = temp_wg_id)
        outcome_word_id_list = temp_wg_id_list[np.all(a = (ls_wchar_matrix - ls_wchar_matrix[temp_wg_id, :]) >= 0, axis = 1)]
                
        n_from_words = outcome_word_id_list.shape[0]
        
        if n_from_words > 0:
            outcome_word_id_list = format_output_list(outcome_word_id_list=outcome_word_id_list, wg_id=curr_wg_id)
            #print(outcome_word_id_list.shape)
            
            # enumerate the from/parent words
            new_anagram_pair_count = anagram_pair_count + n_from_words
            #print(anagram_pair_count, new_anagram_pair_count)

            output_list[anagram_pair_count:new_anagram_pair_count, :] = outcome_word_id_list

            # update the anagram pair count
            anagram_pair_count = new_anagram_pair_count

    curr_time = calc_time(time_start=start_time, round_digits=8)
    output_time_list.append([ls_id, n_search_space, curr_time])

In [ ]:
def build_timing_and_output_objects(output_time_list:list, ls_df:pd.DataFrame):
    
    col_names =['letter_selector_id', 'n_search_space', 'curr_time']
    time_df = pd.DataFrame(data = output_time_list, columns=col_names)
    get_hms(seconds = time_df['curr_time'].sum(),round_seconds_digits=4)
    # join in the other information
    time_df = pd.merge(left = time_df, right = ls_df)

    time_df['avg_lookup_time'] = time_df['curr_time'] / (time_df['ls_count'] * time_df['n_search_space'])

    return time_df   

In [ ]:
time_df = build_timing_and_output_objects(output_time_list=output_time_list,
                                          ls_df = ls_df)

In [ ]:
time_df.head()

In [ ]:
time_df.tail()

In [ ]:
time_df['n_search_space'].describe()

In [ ]:
time_df['ls_count'].sum()

In [ ]:
time_df['avg_lookup_time'].describe()

In [ ]:
print('...truncating output list...')
output_indices = np.all(output_list >= 0, axis=1)
output_list = output_list[output_indices,]
output_list.shape

In [ ]:
print('...building output counters...')
from_word_counter = collections.Counter(output_list[:,1])
to_word_counter = collections.Counter(output_list[:,0])

In [ ]:
# check to see if the word 'acanthology', with word_group_id 746 is in the counter
print(from_word_counter[746])
print(to_word_counter[746])

# Do it with CUPY

In [ ]:
import cupy as cp

In [ ]:
from cupyx.profiler import benchmark

def my_func(a):
    return cp.sqrt(cp.sum(a**2, axis=-1))

a = cp.random.random((256, 1024))
print(benchmark(my_func, (a,), n_repeat=20))  


In [ ]:
# initialize some gpu stats
start_gpu = cp.cuda.Event()
end_gpu = cp.cuda.Event()

start_gpu.record()
start_cpu = time.perf_counter()
out = my_func(a)
end_cpu = time.perf_counter()
end_gpu.record()
end_gpu.synchronize()
t_gpu = cp.cuda.get_elapsed_time(start_gpu, end_gpu)
t_cpu = end_cpu - start_cpu
print(t_gpu, t_cpu)

## create cupy objects

In [ ]:
char_matrix_cp = cp.asarray(a = char_matrix)
word_group_id_list_cp = cp.asarray(a = word_group_id_list)
word_id_list = cp.asarray(a = word_id_list)
wchar_matrix_cp = cp.asarray(a = wchar_matrix)
word_group_id_list_cp = cp.asarray(a = word_group_id_list)

In [ ]:
# the naive approach
# establish counters for record keeping
# sample 1000 rows
n_samples = 1000
sample_wg_id = wg_df['word_group_id'].sample(n = n_samples, random_state = 42)
# sample_wg_id = wg_df['word_group_id'].to_numpy()
sample_wg_id_cp = cp.asarray(a = sample_wg_id)

output_list_cp = cp.full(shape=(n_possible_anagrams, 2),
                          fill_value=-1, dtype=int)

row_count = 0
anagram_pair_count = 0
intermediate_to_word_count = collections.Counter()

for wg_id in sample_wg_id_cp:
    outcome = wchar_matrix_cp - wchar_matrix_cp[wg_id, ]
    
    # compute the score by finding where rows, across all columns, are GTE 0
    outcome_indices_cp = cp.all(outcome >= 0, axis=1)
    outcome = None

    n_from_words = outcome_indices_cp.sum()   
            
    
    if n_from_words >= 1:
        # extract anagrams based on index values    
        outcome_word_id_list_cp = word_group_id_list_cp[outcome_indices_cp]    

        # we have matches
        curr_output_list_cp = cp.zeros(shape=(outcome_word_id_list_cp.shape[0], 2), dtype=int)

        # update the output list with the word_id_list - these are from/parent words
        curr_output_list_cp[:, 0] = outcome_word_id_list_cp

        # update with the word_id - this is the to/child word
        curr_output_list_cp[:, 1] = wg_id

        # enumerate the from/parent wordsds
        new_anagram_pair_count = anagram_pair_count + n_from_words

        # update the total output list
        output_list_cp[anagram_pair_count:new_anagram_pair_count,
                    :] = curr_output_list_cp

        # n_to_word_counter = collections.Counter(output_list[:, 0])
        intermediate_to_word_count.update(outcome_word_id_list_cp.tolist())

        # set the anagram pair count
        anagram_pair_count = new_anagram_pair_count
    
    row_count += 1

    if row_count % 10000 == 0:
        print(row_count)

print('truncating list')
output_indices_cp = cp.all(output_list_cp >= 0, axis=1)
output_list_cp = output_list_cp[output_indices_cp,]
del output_indices_cp
print(output_list_cp.shape)

In [ ]:
def format_output_list_cp(outcome_word_id_list: cp.ndarray, wg_id: int) -> cp.ndarray:
        
    output_list = cp.zeros(shape=(outcome_word_id_list.shape[0], 2), dtype=int)

    # update the output list with the word_id_list - these are from/parent words
    output_list[:, 0] = outcome_word_id_list
    

    # update with the word_id - this is the to/child word
    output_list[:, 1] = wg_id

    return output_list


In [ ]:
# create the output list
output_list_cp = cp.full(shape = (n_possible_anagrams, 2), fill_value=-1)
output_time_list = []

# start counting
anagram_pair_count = 0

#for ls_id_index in range(0, 10):
for ls_row_id, ls_row in ls_df.iloc[:100].iterrows():    
    if ls_row_id % 100 == 0:
        print(ls_row_id)
    start_time = perf_counter_ns()
    
    # get letter selector id information
    ls_id = ls_row['letter_selector_id']
    #ls_id_index = cp.array(ls_row['ls_index'])    
    ls_id_index = ls_row['ls_index']

    ##
    # BUILD A COLUMN SELECTOR
    ##
    # make sure that only values GTE 0 are selected so that the right number of
    # columns are return.
    #curr_ls_id = ls_id_index[ls_id_index >= 0]
    
    ##
    # SUBSET THE wchar_matrix by column selector
    ##    
    outcome_indices_cp = cp.all(wchar_matrix_cp[:, ls_id_index] >= 1, axis=1)
    
    # this is the sub-matrix from which to query
    ls_wchar_matrix_cp = wchar_matrix_cp[outcome_indices_cp, :]
        
    # this is the list of word group ids that correspond to the word group ids
    # in the ls_wchar_matrix
    temp_wg_id_list_cp = word_group_id_list_cp[outcome_indices_cp]
    # place into a dictionary to go from wg_id to wg_index. What is the index
    # of wg_id 675?
    # wg_id_dict = {wg_id:wg_index for wg_index, wg_id in enumerate(temp_wg_id_list)}

    # this is the number of word groups that meet certain criteria. 
    # for example, words that feature the letters: 'bro'    
    n_search_space = temp_wg_id_list_cp.shape[0]
        
    #def my_func(row):
    #    return temp_wg_id_list[np.all(a = (ls_wchar_matrix - ls_wchar_matrix[row, :]) >= 0, axis = 1)]

    #for ii in range(0, ls_wchar_matrix.shape[0]):    
    #for i_curr_wg_id, curr_wg_id in enumerate(temp_wg_id_list):
    # the current list of words featuring the set of least common letters.
    # these are the words have the least common letters of 'bro'    
    curr_wg_id_list = wg_df.loc[wg_df['letter_selector_id'] == ls_id, 'word_group_id'].to_list()
    # n_lookups = curr_wg_id_list.shape[0]
    # n_search_space >= n_lookups, always. 
    for i_curr_wg_id, curr_wg_id in enumerate(curr_wg_id_list):    
        
        #temp_wg_id = wg_id_dict[curr_wg_id]
        temp_wg_id = cp.where(temp_wg_id_list_cp == curr_wg_id)[0][0]
        #print(curr_wg_id, temp_wg_id)

        #outcome_word_id_list = my_func(row = temp_wg_id)
        outcome_word_id_list_cp = temp_wg_id_list_cp[cp.all(a = (ls_wchar_matrix_cp - ls_wchar_matrix_cp[temp_wg_id, :]) >= 0, axis = 1)]
                
        n_from_words = outcome_word_id_list_cp.shape[0]
        
        if n_from_words > 0:
            outcome_word_id_list = format_output_list_cp(outcome_word_id_list=outcome_word_id_list_cp, wg_id=curr_wg_id)
            #print(outcome_word_id_list.shape)
            
            # enumerate the from/parent words
            new_anagram_pair_count = anagram_pair_count + n_from_words
            #print(anagram_pair_count, new_anagram_pair_count)

            output_list_cp[anagram_pair_count:new_anagram_pair_count, :] = outcome_word_id_list

            # update the anagram pair count
            anagram_pair_count = new_anagram_pair_count

    curr_time = calc_time(time_start=start_time, round_digits=8)
    output_time_list.append([ls_id, n_search_space, curr_time])

In [ ]:
col_names =['letter_selector_id', 'n_search_space', 'curr_time']
time_df = pd.DataFrame(data = output_time_list, columns=col_names)

In [ ]:
get_hms(seconds = time_df['curr_time'].sum(),round_seconds_digits=4)

In [ ]:
# join in the other information
time_df = pd.merge(left = time_df, right = ls_df)
time_df['n_search_space'].describe()

In [ ]:
print('...truncating output list...')
output_indices_cp = cp.all(output_list_cp >= 0, axis=1)
output_list_cp = output_list_cp[output_indices_cp,]

output_list_cp.shape

In [ ]:
from_word_counter = collections.Counter(cp.asnumpy(output_list_cp)[:,1])
to_word_counter = collections.Counter(cp.asnumpy(output_list_cp)[:,0])

# check to see if the word 'acanthology', with word_group_id 746 is in the counter
print(from_word_counter[746])
print(to_word_counter[746])
